In [19]:
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [20]:
import pandas as pd
filepath = '/Users/seeni-2328/Documents/Seeni/Datasets/kaggle/lda and t sne interactive visualization/Papers.csv'
p_df = pd.read_csv(filepath)

In [21]:
p_df.dtypes

Id            int64
Title        object
EventType    object
PdfName      object
Abstract     object
PaperText    object
dtype: object

In [22]:
docs = array(p_df['PaperText'])

In [23]:
len(docs)

403

In [24]:
# raw tsne parameters 
p_df.count()

Id           403
Title        403
EventType    403
PdfName      403
Abstract     403
PaperText    403
dtype: int64

In [25]:
# after embedding tsne parameters
p_df.count()

Id           403
Title        403
EventType    403
PdfName      403
Abstract     403
PaperText    403
dtype: int64

In [26]:
p_df.head()

,Id,Title,EventType,PdfName,Abstract,PaperText
0,5677,Double or Nothing: Multiplicative Incentive Me...,Poster,5677-double-or-nothing-multiplicative-incentiv...,Crowdsourcing has gained immense popularity in...,Double or Nothing: Multiplicative\nIncentive M...
1,5941,Learning with Symmetric Label Noise: The Impor...,Spotlight,5941-learning-with-symmetric-label-noise-the-i...,Convex potential minimisation is the de facto ...,Learning with Symmetric Label Noise: The\nImpo...
2,6019,Algorithmic Stability and Uniform Generalization,Poster,6019-algorithmic-stability-and-uniform-general...,One of the central questions in statistical le...,Algorithmic Stability and Uniform Generalizati...
3,6035,Adaptive Low-Complexity Sequential Inference f...,Poster,6035-adaptive-low-complexity-sequential-infere...,We develop a sequential low-complexity inferen...,Adaptive Low-Complexity Sequential Inference f...
4,5978,Covariance-Controlled Adaptive Langevin Thermo...,Poster,5978-covariance-controlled-adaptive-langevin-t...,Monte Carlo sampling for Bayesian posterior in...,Covariance-Controlled Adaptive Langevin\nTherm...


In [27]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]

    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]

    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

    return docs


In [28]:
docs = docs_preprocessor(docs)

In [29]:
print(docs[0][0])

double


In [33]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/usr/local/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


KeyboardInterrupt: 

In [12]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 39534
Number of unique words after removing rare and common words: 6001


In [13]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))


Number of unique tokens: 6001
Number of documents: 403


In [14]:
from gensim.models import LdaModel
# Set training parameters.
num_topics = 4
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)


CPU times: user 35.6 s, sys: 78.5 ms, total: 35.7 s
Wall time: 35.8 s


In [71]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()


In [16]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.176688  0.021628       1        1  29.214605
3     -0.011784 -0.050844       2        1  25.714439
2     -0.071880 -0.080531       3        1  24.998339
0     -0.093024  0.109748       4        1  20.072618, topic_info=     Category         Freq                      Term        Total  loglift  \
term                                                                         
609   Default  1158.000000                    regret  1158.000000  30.0000   
3192  Default   666.000000                    bandit   666.000000  29.0000   
3105  Default   713.000000         matrix_completion   713.000000  28.0000   
3140  Default   679.000000               rank_matrix   679.000000  27.0000   
3185  Default   645.000000           active_learning   645.000000  26.0000   
2180  Default   666.000000             convolutional   666.000000  25.0000   
610   Default   472.000000              regret_bound   472.000000  24.0000   
270   Default   521.000000                    reward   521.000000  23.0000   
123   Default   527.000000                      game   527.000000  22.0000   
2366  Default   552.000000            singular_value   552.000000  21.0000   
2333  Default   569.000000                 recurrent   569.000000  20.0000   
3287  Default   586.000000                submodular   586.000000  19.0000   
7     Default   461.000000                     agent   461.000000  18.0000   
3172  Default   655.000000                    tensor   655.000000  17.0000   
2336  Default   535.000000          recurrent_neural   535.000000  16.0000   
3048  Default   940.000000          convergence_rate   940.000000  15.0000   
1311  Default   541.000000               hidden_unit   541.000000  14.0000   
3937  Default   493.000000              hidden_layer   493.000000  13.0000   
5038  Default   406.000000           reward_function   406.000000  12.0000   
3167  Default   481.000000           strongly_convex   481.000000  11.0000   
2272  Default   475.000000                      lstm   475.000000  10.0000   
3142  Default   476.000000                  recovery   476.000000   9.0000   
2221  Default   588.000000           fully_connected   588.000000   8.0000   
3841  Default   350.000000                    player   350.000000   7.0000   
1143  Default   558.000000           online_learning   558.000000   6.0000   
3612  Default   479.000000                    neuron   479.000000   5.0000   
335   Default   799.000000     variational_inference   799.000000   4.0000   
74    Default   607.000000             deep_learning   607.000000   3.0000   
3152  Default   887.000000         sample_complexity   887.000000   2.0000   
3230  Default   910.000000          gaussian_process   910.000000   1.0000   
...       ...          ...                       ...          ...      ...   
5413   Topic4    56.517315                    payoff    58.446033   1.5723   
3185   Topic4   588.901428           active_learning   645.671143   1.5138   
123    Topic4   483.662140                      game   527.244446   1.5195   
3167   Topic4   425.901459           strongly_convex   481.769562   1.4826   
3226   Topic4    86.936005  exploration_exploitation    92.056305   1.5486   
1607   Topic4   226.453217                     price   256.204041   1.4824   
4139   Topic4   139.021255                   horizon   153.811111   1.5047   
1143   Topic4   443.101044           online_learning   558.780518   1.3739   
1290   Topic4   139.580612               equilibrium   158.254181   1.4803   
3243   Topic4   107.276901                     maker   118.757103   1.5041   
3048   Topic4   561.605103          convergence_rate   940.759277   1.0899   
3166   Topic4   170.618301          strong_convexity   214.161972   1.3785   
1298   Topic4   208.878067                  feedback   277.467896   1.3219   
2090   Topic4   331.371582                   

In [79]:
from sklearn.metrics.pairwise import cosine_similarity

p_df['tokenz'] = docs

docs1 = p_df['tokenz'].apply(lambda l: l[:int0(len(l)/2)])
docs2 = p_df['tokenz'].apply(lambda l: l[int0(len(l)/2):])


In [84]:
corpus1 = [dictionary.doc2bow(doc) for doc in docs1]
corpus2 = [dictionary.doc2bow(doc) for doc in docs2]

# Using the corpus LDA model tranformation
lda_corpus1 = model[corpus1]
lda_corpus2 = model[corpus2]
for i in range(10) :
    print(lda_corpus2[i])

[(0, 0.8982865), (1, 0.10144702)]
[(0, 0.13426296), (3, 0.8653958)]
[(0, 0.7073055), (1, 0.01981192), (2, 0.2726317)]
[(1, 0.20875853), (3, 0.79062504)]
[(0, 0.04207052), (1, 0.35923475), (3, 0.5984264)]
[(0, 0.16116999), (2, 0.83839005)]
[(0, 0.3414898), (1, 0.102716714), (2, 0.19569142), (3, 0.36010203)]
[(1, 0.2121649), (3, 0.78748995)]
[(1, 0.99963224)]
[(0, 0.18993643), (3, 0.8097223)]


In [97]:
from collections import OrderedDict
def get_doc_topic_dist(model, corpus, kwords=False):
    
    '''
    LDA transformation, for each doc only returns topics with non-zero weight
    This function makes a matrix transformation of docs in the topic space.
    '''
    top_dist =[]
    keys = []

    for d in corpus:
        tmp = {i:0 for i in range(num_topics)}
#         print(tmp)
        tmp.update(dict(model[d]))
#         print(dict(model[d]))
#         print(tmp)
        vals = list(OrderedDict(tmp).values())
        top_dist += [array(vals)]
        if kwords:
            keys += [array(vals).argmax()]
#         print("data : {}, top_dist : {}, keys : {}\n".format(d,vals, array(vals).argmax()))
    
#     print("top_dist : {}, keys : {}".format(top_dist,keys))

    return array(top_dist), keys

In [121]:
top_dist1, _ = get_doc_topic_dist(model, lda_corpus1[0:5])
top_dist2, _ = get_doc_topic_dist(model, lda_corpus2[0:5])
lists = []
for c1, c2 in zip(top_dist1, top_dist2) :
    print(c1, c2)
    print(c1.reshape(1, -1),c2.reshape(1, -1))
    print(cosine_similarity(c1.reshape(1, -1), c2.reshape(1, -1))[0][0])
    lists.append(cosine_similarity(c1.reshape(1, -1), c2.reshape(1, -1))[0][0])
    print("\n")

print(lists)
print(mean(lists))

[0.8644456  0.04243042 0.04457616 0.04854782] [0.0348147  0.8720615  0.04457609 0.04854774]
[[0.8644456  0.04243042 0.04457616 0.04854782]] [[0.0348147  0.8720615  0.04457609 0.04854774]]
0.09403847


[0.8644506  0.04242885 0.04457453 0.04854604] [0.03481686 0.04243298 0.87419933 0.04855077]
[[0.8644506  0.04242885 0.04457453 0.04854604]] [[0.03481686 0.04243298 0.87419933 0.04855077]]
0.0961593


[0.86442196 0.0424378  0.04458394 0.04855629] [0.8644476  0.0424298  0.04457552 0.04854713]
[[0.86442196 0.0424378  0.04458394 0.04855629]] [[0.8644476  0.0424298  0.04457552 0.04854713]]
0.99999994


[0.03482955 0.04244843 0.8741536  0.04856845] [0.03482481 0.04244267 0.8741706  0.04856185]
[[0.03482955 0.04244843 0.8741536  0.04856845]] [[0.03482481 0.04244267 0.8741706  0.04856185]]
1.0


[0.03481411 0.04242963 0.8742093  0.04854693] [0.03481476 0.87206125 0.04457618 0.04854785]
[[0.03481411 0.04242963 0.8742093  0.04854693]] [[0.03481476 0.87206125 0.04457618 0.04854785]]
0.10358982


[0.

In [130]:
random_pairs = np.random.randint(0, len(p_df['PaperText']), size=(400, 2))
random_pairs = random_pairs[0:5]
print(random_pairs[0:5])

[[261 389]
 [311  56]
 [ 95  72]
 [304 264]
 [218 230]]


In [132]:
for i in random_pairs[0:5] :
    print(top_dist1[i[0]], top_dist2[i[1]])
    print(top_dist1[i[0]].reshape(1, -1),top_dist2[i[1]].reshape(1, -1))
    print(cosine_similarity(top_dist1[i[0]].reshape(1, -1), top_dist2[i[1]].reshape(1, -1)))
    lists.append(cosine_similarity(top_dist1[i[0]].reshape(1, -1), top_dist2[i[1]].reshape(1, -1)))
    print("\n")

print(lists)
print(mean(lists))

[0.03481508 0.87206006 0.04457658 0.04854828] [0.864446   0.04243029 0.04457604 0.0485477 ]
[[0.03481508 0.87206006 0.04457658 0.04854828]] [[0.864446   0.04243029 0.04457604 0.0485477 ]]
[[0.09403884]]


[0.86440974 0.04244164 0.04458796 0.04856067] [0.03481166 0.8720726  0.04457221 0.04854352]
[[0.86440974 0.04244164 0.04458796 0.04856067]] [[0.03481166 0.8720726  0.04457221 0.04854352]]
[[0.09405042]]


[0.03482349 0.8720291  0.04458735 0.04856   ] [0.03481936 0.8720443  0.04458207 0.04855425]
[[0.03482349 0.8720291  0.04458735 0.04856   ]] [[0.03481936 0.8720443  0.04458207 0.04855425]]
[[1.0000001]]


[0.03482467 0.87202483 0.04458886 0.04856165] [0.03481556 0.8720583  0.0445772  0.04854894]
[[0.03482467 0.87202483 0.04458886 0.04856165]] [[0.03481556 0.8720583  0.0445772  0.04854894]]
[[1.]]


[0.8644259  0.04243658 0.04458265 0.04855488] [0.03482028 0.04243714 0.874187   0.04855553]
[[0.8644259  0.04243658 0.04458265 0.04855488]] [[0.03482028 0.04243714 0.874187   0.04855553]]
[

In [135]:
top_dist1, _ = get_doc_topic_dist(model, lda_corpus1)
top_dist2, _ = get_doc_topic_dist(model, lda_corpus2)

print("Intra similarity: cosine similarity for corresponding parts of a doc(higher is better):")
print(np.mean([cosine_similarity(c1.reshape(1, -1), c2.reshape(1, -1)) for c1,c2 in zip(top_dist1, top_dist2)]))

random_pairs = np.random.randint(0, len(p_df['PaperText']), size=(400, 2))

print("Inter similarity: cosine similarity between random parts (lower is better):")
print(np.mean([cosine_similarity(top_dist1[i[0]].reshape(1, -1), top_dist2[i[1]].reshape(1, -1)) for i in random_pairs]))

Intra similarity: cosine similarity for corresponding parts of a doc(higher is better):
0.8527726
Inter similarity: cosine similarity between random parts (lower is better):
0.43492416


In [95]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [138]:
topic_summaries = []
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print(tmp[:5])
    topic_summaries += [tmp[:5]]
    print


term                 frequency

Topic 0 |---------------------

regret               0.013
bandit               0.008
active_learning      0.007
convergence_rate     0.006
reward               0.006
game                 0.005
regret_bound         0.005
online_learning      0.005
agent                0.005
strongly_convex      0.005
Topic 1 |---------------------

gaussian_process     0.006
convolutional        0.005
generative_model     0.005
recurrent            0.004
deep_learning        0.004
fully_connected      0.004
recurrent_neural     0.004
hidden_unit          0.004
hidden_layer         0.004
lstm                 0.004
Topic 2 |---------------------

matrix_completion    0.006
rank_matrix          0.006
tensor               0.005
singular_value       0.005
data_set             0.005
sample_complexity    0.004
recovery             0.004
covariance_matrix    0.004
line_search          0.003
nuclear_norm         0.003
Topic 3 |---------------------

variational_inference 0.005
ma

In [23]:
top_labels = {0: 'Statistics', 1:'Numerical Analysis', 2:'Online Learning', 3:'Deep Learning'}

In [24]:
import re
import nltk

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

def paper_to_wordlist( paper, remove_stopwords=True ):
    '''
        Function converts text to a sequence of words,
        Returns a list of words.
    '''
    lemmatizer = WordNetLemmatizer()
    # 1. Remove non-letters
    paper_text = re.sub("[^a-zA-Z]"," ", paper)
    # 2. Convert words to lower case and split them
    words = paper_text.lower().split()
    # 3. Remove stop words
    words = [w for w in words if not w in stops]
    # 4. Remove short words
    words = [t for t in words if len(t) > 2]
    # 5. lemmatizing
    words = [nltk.stem.WordNetLemmatizer().lemmatize(t) for t in words]

    return(words)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvectorizer = TfidfVectorizer(input='content', analyzer = 'word', lowercase=True, stop_words='english',\
                                  tokenizer=paper_to_wordlist, ngram_range=(1, 3), min_df=40, max_df=0.20,\
                                  norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True)

dtm = tvectorizer.fit_transform(p_df['PaperText']).toarray()

/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  sorted(inconsistent))


In [26]:
top_dist =[]
for d in corpus:
    tmp = {i:0 for i in range(num_topics)}
    tmp.update(dict(model[d]))
    vals = list(OrderedDict(tmp).values())
    top_dist += [array(vals)]

In [27]:
top_dist, lda_keys= get_doc_topic_dist(model, corpus, True)
features = tvectorizer.get_feature_names()

In [28]:
top_ws = []
for n in range(len(dtm)):
    inds = int0(argsort(dtm[n])[::-1][:4])
    tmp = [features[i] for i in inds]
    
    top_ws += [' '.join(tmp)]
    
p_df['Text_Rep'] = pd.DataFrame(top_ws)
p_df['clusters'] = pd.DataFrame(lda_keys)
p_df['clusters'].fillna(10, inplace=True)

cluster_colors = {0: 'blue', 1: 'green', 2: 'yellow', 3: 'red', 4: 'skyblue', 5:'salmon', 6:'orange', 7:'maroon', 8:'crimson', 9:'black', 10:'gray'}

p_df['colors'] = p_df['clusters'].apply(lambda l: cluster_colors[l])

In [29]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(top_dist)

In [30]:
p_df['X_tsne'] =X_tsne[:, 0]
p_df['Y_tsne'] =X_tsne[:, 1]

In [37]:
from bokeh.plotting import figure, show, output_notebook, save#, output_file
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [38]:
source = ColumnDataSource(dict(
    x=p_df['X_tsne'],
    y=p_df['Y_tsne'],
    color=p_df['colors'],
    label=p_df['clusters'].apply(lambda l: top_labels[l]),
#     msize= p_df['marker_size'],
    topic_key= p_df['clusters'],
    title= p_df[u'Title'],
    content = p_df['Text_Rep']
))

In [39]:
title = 'T-SNE visualization of topics'

plot_lda = figure(plot_width=1000, plot_height=600,
                     title=title, tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y', legend='label', source=source,
                 color='color', alpha=0.8, size=10)#'msize', )

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "Title: @title, KeyWords: @content - Topic: @topic_key "}
plot_lda.legend.location = "top_left"

show(plot_lda)

#save the plot
# save(plot_lda, '{}.html'.format(title))